In [3]:
import pandas as pd
import geopandas as gpd


In [14]:
# Load the crime dataset and remove crimes that are not of immediate interest
crime_df = pd.read_csv("/Users/someshsahu/Documents/DSAI Project/crimes.csv")

crime_counts = crime_df['OFNS_DESC'].value_counts()
filtered_df = crime_df[crime_df['OFNS_DESC'].isin(crime_counts[crime_counts > 1000].index)]
print(filtered_df.columns)



removed_filtered_df = filtered_df.drop(filtered_df[filtered_df['OFNS_DESC'].isin(['VOIDED ARRESTS', 'OTHER STATE LAWS', 'FORGERY', 'FRAUDS', 'OFFENSES INVOLVING FRAUD', 'INTOXICATED & IMPAIRED DRIVING', 'OTHER TRAFFIC INFRACTION', 'INTOXICATED/IMPAIRED DRIVING', 'GAMBLING', 'THEFT-FRAUD', 'MOVING INFRACTIONS', 'ADMINISTRATIVE CODE', 'OTHER STATE LAWS (NON PENAL LA'])].index)




Index(['ARREST_KEY', 'ARREST_DATE', 'PD_CD', 'PD_DESC', 'KY_CD', 'OFNS_DESC',
       'LAW_CODE', 'LAW_CAT_CD', 'ARREST_BORO', 'ARREST_PRECINCT',
       'JURISDICTION_CODE', 'AGE_GROUP', 'PERP_SEX', 'PERP_RACE', 'X_COORD_CD',
       'Y_COORD_CD', 'Latitude', 'Longitude'],
      dtype='object')


,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude
1,172945127,12/25/2017,153.0,RAPE 3,104.0,RAPE,PL 1302501,F,Q,115,0,25-44,M,WHITE HISPANIC,1018699,215043,40.756861,-73.875657
2,172875607,12/21/2017,173.0,"SEXUAL ABUSE 3,2",233.0,SEX CRIMES,PL 1305200,M,Q,114,97,25-44,F,WHITE,1007654,219564,40.769306,-73.915508
3,172791367,12/20/2017,177.0,NaN,116.0,SEX CRIMES,PL 1306504,F,B,41,0,25-44,M,WHITE,1013232,236725,40.816392,-73.895296
4,172727937,12/18/2017,175.0,"SEXUAL ABUSE 3,2",233.0,SEX CRIMES,PL 13052A1,M,Q,111,0,<18,M,WHITE,1048565,216315,40.760185,-73.767843
5,172744565,12/18/2017,153.0,RAPE 3,104.0,RAPE,PL 1302501,F,Q,105,0,25-44,M,BLACK,1057636,203954,40.726187,-73.735234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1722089,88364151,01/01/2013,490.0,"STOLEN PROPERTY 3,POSSESSION",232.0,POSSESSION OF STOLEN PROPERTY,PL 1654000,M,B,42,0,25-44,M,BLACK,1009444,241764,40.830234,-73.908963
1722090,88364137,01/01/2013,478.0,"THEFT OF SERVICES, UNCLASSIFIED",343.0,OTHER OFFENSES RELATED TO THEF,PL 1651503,M,B,49,0,18-24,M,BLACK,1027434,251387,40.856577,-73.843894
1722091,88364124,01/01/2013,101.0,ASSAULT 3,344.0,ASSAULT 3 & RELATED OFFENSES,PL 1200001,M,B,43,0,25-44,F,WHITE HISPANIC,1018438,239937,40.825189,-73.876472
1722092,88358728,01/01/2013,101.0,ASSAULT 3,344.0,ASSAULT 3 & RELATED OFFENSES,PL 1200001,M,S,120,0,18-24,M,WHITE,965000,162901,40.613789,-74.069336
